Analysing who is healed and when by each medic.

In [ ]:
## Imports

# The library generated by rust
# TODO: ent.PlayerState is not recognized, but ent.Player is ?
import tf2dal as dal
from tf2dal import entities as ent, events as evt, demo, math, game

# If any of these fail, use "!pip install <package>" to install via pip
import numpy as np
#import pyarrow # for pandas 3.0
import pandas as pd
import matplotlib.pyplot as plt
import kmapper as km

In [ ]:
## Logging

import logging
LOGGING_FMT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=LOGGING_FMT)
logging.getLogger().setLevel(logging.INFO)

Information for `demos/demofile.dem`:
- Acquired from [demos.tf/1042159](https://demos.tf/1042159) 
- Associated log at [logs.tf/3501742](https://logs.tf/3501742)

Round data for demofile.dem should be approximately as follows (from log):

| Round | Time | Kills | Winner |
|-------|------|-------|--------|
| 0     | 3:37 | 19    | Red    |
| 1     | 2:41 | 21    | Red    |
| 2     | 8:23 | 47    | Blue   |
| 3     | 2:05 | 9     | Red    |
| 4     | 2:17 | 23    | Red    |
| 5     | 4:34 | 33    | Red    |

In [ ]:
rounds_data = dal.load_demo_rounds("demos/demofile.dem")
print(f"Loaded {len(rounds_data)} round(s)")

for i, rnd in enumerate(rounds_data):
    time = 0.0
    for tick, state in rnd.tick_states.items():
        time += state.tick_delta
    print("Round %d: %2d:%5.2f (%f) | %2d Kills | %5d States | %5d Ticks | Winner: %s" % (
        i, int(np.floor(time/60)), time - np.floor(time/60) * 60, time,
        len(rnd.kills),
        len(rnd.tick_states),
        rnd.rounds[0].end_tick - rnd.rounds[0].start_tick,
        rnd.rounds[0].winner
    ))

Possible table values

- Is Medic
- Being Healed, No Critheals
- Being Healed, With Critheals
- Not Being Healed, No Critheals
- Not Being Healed, With Critheals
- Dead
- No Medic Alive

In [ ]:
def critheal_percent(tslh):
    # 1 = has full critheals. 0 = no crit heals.
    return max(0, min((tslh - 10.0) / 5.0, 1))

def heal_rate(critheal_perc):
    # 24hp/s base, 72 hp/s when fully at crit heals
    return 48 * critheal_perc + 24

# [(heal status tick 1), (tick 2), ...]
round_red_heal_statuses = []
round_blu_heal_statuses = []

IS_MEDIC = 0
HEALED_NO_CRITS = 1
HEALED_CRITS = 2
NOT_HEALED_NO_CRITS = 3
NOT_HEALED_CRITS = 4
DEAD = 5
NO_MEDIC = 6

def get_healstates(tick, data: demo.TickData):
    # Return:
    # ((tick, red player 1, red player 2, ...), (tick blue player 1, blue player 2, ...))

    red = [player for player in data.players if player.team == game.Team.Red]
    blu = [player for player in data.players if player.team == game.Team.Blue]

    i = 0
    ret = []
    for team in [red, blu]:
        i += 1
        team_statuses = [tick]
        medics = [p for p in team if p.player_class == game.Class.Medic]
        if len(medics) == 0 or True not in [m.state == 0 for m in medics]:
            print(f"({i}) medic dead")
            team_statuses.extend([NO_MEDIC for _ in team])
            ret.append(team_statuses)
            continue

        for player in team:
            if player.state != 0:
                #print(f"({i}) +1 dead player")
                team_statuses.append(DEAD)
                continue
            if player.player_class == game.Class.Medic:
                #print(f"({i}) +1 medic")
                team_statuses.append(IS_MEDIC)
                continue
            
            has_crits = critheal_percent(player.time_since_last_hurt) > 0
            being_healed = False
            for medigun in data.mediguns:
                if player.info.entity_id == medigun.heal_target and medigun.is_healing and not medigun.is_holstered:
                    being_healed = True
                    #print(f"({i}) +1 being healed")
                    team_statuses.append(HEALED_CRITS if has_crits else HEALED_NO_CRITS)
                    break
            
            if not being_healed:
                #print(f"({i}) +1 not being healed")
                team_statuses.append(NOT_HEALED_CRITS if has_crits else NOT_HEALED_NO_CRITS)
            
        ret.append(team_statuses)
    
    return ret


for (tick, data) in rounds_data[0].tick_states.items():
    statuses = get_healstates(tick, data)
    if len(statuses[0]) != 7 or len(statuses[1]) != 7:
        print(f"{len(statuses[0])}, {len(statuses[1])}")
        
    round_red_heal_statuses.append(statuses[0])
    round_blu_heal_statuses.append(statuses[1])

   

In [ ]:
blu_byplayer = np.sort(np.asarray(round_blu_heal_statuses), axis=0).transpose()
red_byplayer = np.sort(np.asarray(round_red_heal_statuses), axis=0).transpose()

In [ ]:
total_ticks = len(round_red_heal_statuses)

for i in range(1, 7):
    num_medic = 0
    num_dead = 0
    num_crithealed = 0
    num_slowhealed = 0
    num_critswasted = 0
    num_normal = 0
    num_nomedic = 0

    for status in gobyplayer[i]:
        if status == IS_MEDIC:
            num_medic += 1
        if status == DEAD:
            num_dead += 1
        if status == HEALED_CRITS:
            num_crithealed += 1
        if status == HEALED_NO_CRITS:
            num_slowhealed += 1
        if status == NOT_HEALED_NO_CRITS:
            num_normal += 1
        if status == NOT_HEALED_CRITS:
            num_critswasted += 1
        if status == NO_MEDIC:
            num_nomedic += 1

    print(f"Blu Player {i}:")
    print(f"- Ticks as Medic    : {num_medic} ({float(num_medic*100) / total_ticks}%)")
    print(f"- Ticks w/ No Medic : {num_nomedic} ({float(num_nomedic*100) / total_ticks}%)")
    print(f"- Ticks Crit Healed : {num_crithealed} ({float(num_crithealed*100) / total_ticks}%)")
    print(f"- Ticks Slow Healed : {num_slowhealed} ({float(num_slowhealed*100) / total_ticks}%)")
    print(f"- Ticks Crit Wasted : {num_critswasted} ({float(num_critswasted*100) / total_ticks}%)")
    print(f"- Ticks Normal      : {num_normal} ({float(num_normal*100) / total_ticks}%)")
    print(f"- Ticks Dead        : {num_dead} ({float(num_dead*100) / total_ticks}%)")

In [ ]:
category_names = [
    "Playing Medic",
    "Slow Healed",
    "Crit Healed",
    "Normal",
    "Crit Wasted",
    "Dead"
    "No Medic",
]

def create_results(teamresults):
    ret = {}
    for i in range(1,7):
        uniq = np.unique(teamresults[i], return_counts=True)
        print(f"{uniq[0]}, {uniq[1]}")

        ptitle = f"Player {i}"
        ret[ptitle] = [0 for _ in range(0, 7)]
        for n, j in enumerate(uniq[0]):
            ret[ptitle][j] = round(float(uniq[1][n]) * 100 / total_ticks, 2)
    return ret

red_results = create_results(red_byplayer)
blu_results = create_results(blu_byplayer)

In [ ]:
blu_results

In [ ]:
def survey(results, category_names):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    category_colors = plt.colormaps['RdYlGn'](
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(figsize=(9.2, 5))
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(category_names, category_colors)):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        rects = ax.barh(labels, widths, left=starts, height=0.5,
                        label=colname, color=color)

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'
        ax.bar_label(rects, label_type='center', color=text_color)
    ax.legend(ncols=len(category_names), bbox_to_anchor=(0, 1),
              loc='lower left', fontsize='small')

    return fig, ax

survey(blu_results, category_names)
plt.show()